# Learning to Rank на основе LETOR MQ2008
## Анализ данных, построение и оценка ранжирующей модели




In [96]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("taoqin/letor4")

# print("Path to dataset files:", path)

In [97]:
import os
import pandas as pd
import numpy as np
from lightgbm import LGBMModel, Dataset, train
from sklearn.metrics import ndcg_score
import optuna
from tqdm import tqdm

In [98]:
fold_path = "/kaggle/input/letor4/MQ2008/Fold1"


print("Проверка наличия файлов:")
print("train.txt:", os.path.isfile(os.path.join(fold_path, "train.txt")))
print("vali.txt:", os.path.isfile(os.path.join(fold_path, "vali.txt")))
print("test.txt:", os.path.isfile(os.path.join(fold_path, "test.txt")))

Проверка наличия файлов:
train.txt: True
vali.txt: True
test.txt: True


In [99]:
def load_letor_data(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split('#')[0].split()
            label = int(parts[0])
            qid = int(parts[1].split(':')[1])
            features = {f'feature_{i}': 0.0 for i in range(1, 47)}

            for elem in parts[2:]:
                if ':' in elem:
                    fid, value = elem.split(':')
                    fid = int(fid)
                    if 1 <= fid <= 46:
                        features[f'feature_{fid}'] = float(value)
                    else:
                      print(f"Недопуститмый признак. elem: {elem}, fid: {fid}" )

            row = {'label': label, 'qid': qid}
            row.update(features)
            data.append(row)

    df = pd.DataFrame(data)
    return df

In [100]:
train_df = load_letor_data(os.path.join(fold_path, "train.txt"))
print(f"Тренировочный сет: {train_df.head(5)}")
print(f"Размер: {train_df.shape}")

vali_df = load_letor_data(os.path.join(fold_path, "vali.txt"))
print(f"Валидационный сет: {vali_df.head(5)}")
print(f"Размер: {vali_df.shape}")

test_df = load_letor_data(os.path.join(fold_path, "test.txt"))
print(f"Тестовый сет: {test_df.head(5)}")
print(f"Размер: {test_df.shape}")

Тренировочный сет:    label    qid  feature_1  feature_2  feature_3  feature_4  feature_5  \
0      0  10002   0.007477        0.0        1.0        0.0   0.007470   
1      0  10002   0.603738        0.0        1.0        0.0   0.603175   
2      0  10002   0.214953        0.0        0.0        0.0   0.213819   
3      0  10002   0.000000        0.0        1.0        0.0   0.000000   
4      0  10002   1.000000        1.0        0.0        0.0   1.000000   

   feature_6  feature_7  feature_8  ...  feature_37  feature_38  feature_39  \
0        0.0        0.0        0.0  ...    0.797056    0.697327    0.721953   
1        0.0        0.0        0.0  ...    0.000000    0.000000    0.117399   
2        0.0        0.0        0.0  ...    0.566409    0.760916    0.746370   
3        0.0        0.0        0.0  ...    0.320586    0.133604    0.000000   
4        0.0        0.0        0.0  ...    0.341228    0.292567    0.385569   

   feature_40  feature_41  feature_42  feature_43  feature_44

##EDA


In [101]:
train_df['label'].value_counts()

,count
label,
0,7820
1,1223
2,587


In [102]:
vali_df['label'].value_counts()

,count
label,
0,2140
1,400
2,167


In [103]:
test_df['label'].value_counts()

,count
label,
0,2319
1,378
2,177


Наблюдается дисбаланс классов - класс 0 преобладает над остальными. Логично для специфики задачи - для конкретного запроса во всем интернете найдется больше нерелевантных документов, чем релевантных

In [104]:
train_label_stats = train_df.groupby('qid')['label'].agg(['max']).value_counts()
train_label_stats

,count
max,
2,200
1,139
0,132


In [105]:
vali_label_stats = vali_df.groupby('qid')['label'].agg(['size', 'max']).describe()
vali_label_stats

,size,max
count,157.000000,157.000000
mean,17.242038,1.197452
std,17.278414,0.796140
min,6.000000,0.000000
25%,8.000000,1.000000
50%,8.000000,1.000000
75%,16.000000,2.000000
max,118.000000,2.000000


In [106]:
test_label_stats = test_df.groupby('qid')['label'].agg(['max']).value_counts()
test_label_stats

,count
max,
2,63
0,51
1,42


Можно увидеть, что для некотрых запросов отсутсвуют релевантные документы и в таком случае, задача ранжирования для них неактуальна, однако не стоит удалять эти запросы из наших данных так, как в релаьном мире пользователи также могут оставлять некорректные запросы и наша цель - сформировать хоть какую-то выдачу. Также если мы удалим такие запросы, то поднимем итоговые метрики, однако они будут неактуальны потому, что, как я и сказал раньше, при использовании могут приходить "плохие данные", с которыми му не будем знать, что делать.


Также по анализу валидационной выборки можно увидеть разброс в количестве документов для разных запросов (от 6 до 118). Это может негативно сказаться на обучении, ведь тогда модель будет уделять внимание наибольшим группам и не уловить зависимости для маленьких. Для минимизации этой проблемы предлагается мониторить качество не только в целом, но и по подвыборкам разного размера.


#Обучение

In [107]:
group_train = train_df.sort_values(by="qid").groupby("qid").size().tolist()
group_vali = vali_df.sort_values(by="qid").groupby("qid").size().tolist()
group_test = test_df.sort_values(by="qid").groupby("qid").size().tolist()

In [108]:
x_train = train_df.drop(['label', 'qid'], axis=1)
y_train = train_df['label']
x_vali = vali_df.drop(['label', 'qid'], axis=1)
y_vali = vali_df['label']

In [109]:
train_data = Dataset(x_train, label=y_train, group=group_train)
vali_data = Dataset(x_vali, label=y_vali, group=group_vali, reference=train_data)

In [110]:
params = {
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'eval_at': [5, 10],
    'learning_rate': 0.05,
    'verbose': 1,
    'early_stopping_rounds': 50
}

model = train(params,
                  train_data,
                  valid_sets=[vali_data],
                  valid_names=['vali'],
                  num_boost_round=500)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008247 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9233
[LightGBM] [Info] Number of data points in the train set: 9630, number of used features: 40
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[16]	vali's ndcg@5: 0.747068	vali's ndcg@10: 0.793359


In [111]:
x_vali = vali_df.drop(['label', 'qid'], axis=1)
y_vali = vali_df['label']
preds_vali = model.predict(x_vali)

qid_vali = vali_df['qid']
vali_results_df = pd.DataFrame({
    'qid': qid_vali,
    'true_label': y_vali,
    'pred_score': preds_vali
})

results_list = []

for qid, group in vali_results_df.groupby('qid'):
    y_true = group['true_label'].values
    y_pred = group['pred_score'].values
    num_docs = len(y_true)
    if num_docs > 1:
        try:
            ndcg_5 = ndcg_score([y_true], [y_pred], k=5)
            ndcg_10 = ndcg_score([y_true], [y_pred], k=10)
        except:
            ndcg_5, ndcg_10 = np.nan, np.nan
    else:
        ndcg_5, ndcg_10 = np.nan, np.nan

    max_label = np.max(y_true)
    min_label = np.min(y_true)
    sum_relevant = np.sum(y_true > 0)

    results_list.append({
        'qid': qid,
        'num_docs': num_docs,
        'max_label': max_label,
        'min_label': min_label,
        'sum_relevant': sum_relevant,
        'ndcg@5': ndcg_5,
        'ndcg@10': ndcg_10
    })

query_results_df = pd.DataFrame(results_list)

print("=== ОБЩАЯ СТАТИСТИКА ПО ВАЛИДАЦИИ ===")
print(f"Всего запросов: {len(query_results_df)}")
print(f"Vali NDCG@5: {query_results_df['ndcg@5'].mean():.4f}")
print(f"Vali NDCG@10: {query_results_df['ndcg@10'].mean():.4f}")
print()

print("=== РАСПРЕДЕЛЕНИЕ NDCG ПО ЗАПРОСАМ ===")
print("NDCG@5:")
print(query_results_df['ndcg@5'].describe())
print("\nNDCG@10:")
print(query_results_df['ndcg@10'].describe())
print()

print("=== АНАЛИЗ ПО КАЧЕСТВУ ЗАПРОСОВ ===")
print("\nЗапросы БЕЗ релевантных документов (max_label = 0):")
no_relevant = query_results_df[query_results_df['max_label'] == 0]
print(f"Количество: {len(no_relevant)}")
print(f"Средний NDCG@5: {no_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {no_relevant['ndcg@10'].mean():.4f}")

print("\nЗапросы С релевантными документами (max_label > 0):")
has_relevant = query_results_df[query_results_df['max_label'] > 0]
print(f"Количество: {len(has_relevant)}")
print(f"Средний NDCG@5: {has_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {has_relevant['ndcg@10'].mean():.4f}")

print("\nЗапросы с высокорелевантными документами (max_label = 2):")
has_high_relevant = query_results_df[query_results_df['max_label'] == 2]
print(f"Количество: {len(has_high_relevant)}")
print(f"Средний NDCG@5: {has_high_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {has_high_relevant['ndcg@10'].mean():.4f}")


=== ОБЩАЯ СТАТИСТИКА ПО ВАЛИДАЦИИ ===
Всего запросов: 157
Vali NDCG@5: 0.5200
Vali NDCG@10: 0.5650

=== РАСПРЕДЕЛЕНИЕ NDCG ПО ЗАПРОСАМ ===
NDCG@5:
count    157.000000
mean       0.520011
std        0.383589
min        0.000000
25%        0.000000
50%        0.591089
75%        0.885460
max        1.000000
Name: ndcg@5, dtype: float64

NDCG@10:
count    157.000000
mean       0.565035
std        0.380661
min        0.000000
25%        0.084299
50%        0.669977
75%        0.906025
max        1.000000
Name: ndcg@10, dtype: float64

=== АНАЛИЗ ПО КАЧЕСТВУ ЗАПРОСОВ ===

Запросы БЕЗ релевантных документов (max_label = 0):
Количество: 37
Средний NDCG@5: 0.0000
Средний NDCG@10: 0.0000

Запросы С релевантными документами (max_label > 0):
Количество: 120
Средний NDCG@5: 0.6803
Средний NDCG@10: 0.7393

Запросы с высокорелевантными документами (max_label = 2):
Количество: 68
Средний NDCG@5: 0.6980
Средний NDCG@10: 0.7466


In [112]:
final_ndcg_5_vali = query_results_df[query_results_df['max_label'] > 0]['ndcg@5'].mean()
final_ndcg_10_vali = query_results_df[query_results_df['max_label'] > 0]['ndcg@10'].mean()

print(f"Финальный NDCG@5 на валидации (по релевантным запросам): {final_ndcg_5_vali:.4f}")
print(f"Финальный NDCG@10 на валидации (по релевантным запросам): {final_ndcg_10_vali:.4f}")

Финальный NDCG@5 на валидации (по релевантным запросам): 0.6803
Финальный NDCG@10 на валидации (по релевантным запросам): 0.7393


Две верхние ячейки были добавлены после рассчета NDCG для тестовой выборке, где он оказался сильно хуже чем для валидации, это натолкнуло меня на мысль, что LightGBM использует другую технику подсчета NDCG, после просмотра исходного кода оказалось, что для запросов без релевантных документов он устанавливает значение в 1, что сильно преувеличивает качество модели, после этого было принято решение считать NDCG вручную и устанавливать 0 для запросов без релевантных документов

#Тест

In [113]:
importance_gain = model.feature_importance(importance_type='gain')

feature_names = x_train.columns
feat_imp_df = pd.DataFrame({
    'feature': feature_names,
    'split_importance': model.feature_importance(importance_type='split'),
    'gain_importance': model.feature_importance(importance_type='gain')
})

feat_imp_df = feat_imp_df.sort_values('gain_importance', ascending=False)
feat_imp_df.head(10)

,feature,split_importance,gain_importance
38,feature_39,43,1870.566962
39,feature_40,28,1595.054274
22,feature_23,8,324.487775
21,feature_22,31,316.704690
23,feature_24,22,250.531600
28,feature_29,21,248.080163
18,feature_19,27,228.706899
15,feature_16,23,213.759079
41,feature_42,16,195.665241
44,feature_45,18,182.239460


In [114]:
x_test = test_df.drop(['label', 'qid'], axis=1)
y_test = test_df['label']

In [115]:
preds = model.predict(x_test)

In [116]:
qid_test = test_df['qid']
test_results_df = pd.DataFrame({
    'qid': qid_test,
    'true_label': y_test,
    'pred_score': preds
})

results_list = []

for qid, group in test_results_df.groupby('qid'):
    y_true = group['true_label'].values
    y_pred = group['pred_score'].values
    num_docs = len(y_true)
    if num_docs > 1:
        try:
            ndcg_5 = ndcg_score([y_true], [y_pred], k=5)
            ndcg_10 = ndcg_score([y_true], [y_pred], k=10)
        except:
            ndcg_5, ndcg_10 = np.nan, np.nan
    else:
        ndcg_5, ndcg_10 = np.nan, np.nan

    max_label = np.max(y_true)
    min_label = np.min(y_true)
    sum_relevant = np.sum(y_true > 0)

    results_list.append({
        'qid': qid,
        'num_docs': num_docs,
        'max_label': max_label,
        'min_label': min_label,
        'sum_relevant': sum_relevant,
        'ndcg@5': ndcg_5,
        'ndcg@10': ndcg_10
    })

query_results_df = pd.DataFrame(results_list)

print("=== ОБЩАЯ СТАТИСТИКА ПО ТЕСТУ ===")
print(f"Всего запросов: {len(query_results_df)}")
print(f"Test NDCG@5: {query_results_df['ndcg@5'].mean():.4f}")
print(f"Test NDCG@10: {query_results_df['ndcg@10'].mean():.4f}")
print()

print("=== РАСПРЕДЕЛЕНИЕ NDCG ПО ЗАПРОСАМ ===")
print("NDCG@5:")
print(query_results_df['ndcg@5'].describe())
print("\nNDCG@10:")
print(query_results_df['ndcg@10'].describe())
print()

print("=== АНАЛИЗ ПО КАЧЕСТВУ ЗАПРОСОВ ===")
print("\nЗапросы БЕЗ релевантных документов (max_label = 0):")
no_relevant = query_results_df[query_results_df['max_label'] == 0]
print(f"Количество: {len(no_relevant)}")
print(f"Средний NDCG@5: {no_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {no_relevant['ndcg@10'].mean():.4f}")

print("\nЗапросы С релевантными документами (max_label > 0):")
has_relevant = query_results_df[query_results_df['max_label'] > 0]
print(f"Количество: {len(has_relevant)}")
print(f"Средний NDCG@5: {has_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {has_relevant['ndcg@10'].mean():.4f}")

print("\nЗапросы с высокорелевантными документами (max_label = 2):")
has_high_relevant = query_results_df[query_results_df['max_label'] == 2]
print(f"Количество: {len(has_high_relevant)}")
print(f"Средний NDCG@5: {has_high_relevant['ndcg@5'].mean():.4f}")
print(f"Средний NDCG@10: {has_high_relevant['ndcg@10'].mean():.4f}")


=== ОБЩАЯ СТАТИСТИКА ПО ТЕСТУ ===
Всего запросов: 156
Test NDCG@5: 0.4679
Test NDCG@10: 0.4981

=== РАСПРЕДЕЛЕНИЕ NDCG ПО ЗАПРОСАМ ===
NDCG@5:
count    156.000000
mean       0.467921
std        0.403612
min        0.000000
25%        0.000000
50%        0.543595
75%        0.867514
max        1.000000
Name: ndcg@5, dtype: float64

NDCG@10:
count    156.000000
mean       0.498067
std        0.398331
min        0.000000
25%        0.000000
50%        0.595629
75%        0.876508
max        1.000000
Name: ndcg@10, dtype: float64

=== АНАЛИЗ ПО КАЧЕСТВУ ЗАПРОСОВ ===

Запросы БЕЗ релевантных документов (max_label = 0):
Количество: 51
Средний NDCG@5: 0.0000
Средний NDCG@10: 0.0000

Запросы С релевантными документами (max_label > 0):
Количество: 105
Средний NDCG@5: 0.6952
Средний NDCG@10: 0.7400

Запросы с высокорелевантными документами (max_label = 2):
Количество: 63
Средний NDCG@5: 0.7591
Средний NDCG@10: 0.7941


Можно увидеть, что треть всех запросов не имеют релевантнвых документов, соответсвтенно их ndcg портит общую статистику исходя из этого факта было рпешено удалить такие документы из оценик качества.

In [117]:
final_ndcg_5_test = query_results_df[query_results_df['max_label'] > 0]['ndcg@5'].mean()
final_ndcg_10_test = query_results_df[query_results_df['max_label'] > 0]['ndcg@10'].mean()

print(f"Финальный NDCG@5 на тесте (по релевантным запросам): {final_ndcg_5_test:.4f}")
print(f"Финальный NDCG@10 на тесте (по релевантным запросам): {final_ndcg_10_test:.4f}")

Финальный NDCG@5 на тесте (по релевантным запросам): 0.6952
Финальный NDCG@10 на тесте (по релевантным запросам): 0.7400


#Hyperparameter tuning

In [118]:
def create_datasets_with_params(feature_pre_filter=True):

    train_data = Dataset(x_train, label=y_train, group=group_train,
                        params={'feature_pre_filter': feature_pre_filter})
    vali_data = Dataset(x_vali, label=y_vali, group=group_vali,
                       params={'feature_pre_filter': feature_pre_filter},
                       reference=train_data)
    return train_data, vali_data

def compute_fair_ndcg(y_true, y_pred, qids, k=5):
    ndcg_scores = []

    for qid in np.unique(qids):
        mask = qids == qid
        q_true = y_true[mask]
        q_pred = y_pred[mask]

        if len(q_true) > 1 and np.max(q_true) > 0:
            try:
                ndcg = ndcg_score([q_true], [q_pred], k=k)
                if not np.isnan(ndcg):
                    ndcg_scores.append(ndcg)
            except:
                continue

    return np.mean(ndcg_scores) if ndcg_scores else 0

def objective(trial):
    params = {
        'objective': 'lambdarank',
        'metric': 'ndcg',
        'eval_at': [5],
        'verbosity': -1,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 100),
        'max_depth': trial.suggest_int('max_depth', 4, 8),
        'min_child_samples': trial.suggest_int('min_child_samples', 20, 100),
        'subsample': trial.suggest_float('subsample', 0.7, 0.9),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.7, 0.9),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 1.0, log=True),
        'early_stopping_rounds' : 20
    }

    train_data_optuna, vali_data_optuna = create_datasets_with_params(feature_pre_filter=False)

    model = train(
        params,
        train_data_optuna,
        valid_sets=[vali_data_optuna],
        valid_names=['vali'],
        num_boost_round=200,
    )

    vali_preds = model.predict(x_vali, num_iteration=model.best_iteration)

    ndcg_scores = []
    for qid in vali_df['qid'].unique():
        group_mask = vali_df['qid'] == qid
        y_true = vali_df.loc[group_mask, 'label'].values
        y_pred = vali_preds[group_mask]

        # Только группы с релевантными документами
        if len(y_true) > 1 and np.max(y_true) > 0:
            try:
                ndcg = ndcg_score([y_true], [y_pred], k=5)
                if not np.isnan(ndcg):
                    ndcg_scores.append(ndcg)
            except:
                continue

    return 1 - np.mean(ndcg_scores) if ndcg_scores else 1.0

study = optuna.create_study(direction='minimize')

print("Запуск оптимизации гиперпараметров...")
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("\n=== ЛУЧШИЕ ГИПЕРПАРАМЕТРЫ ===")
print(f"Лучшее значение (1 - NDCG@5): {study.best_value:.4f}")
print(f"Лучший NDCG@5: {1 - study.best_value:.4f}")
print(f"Лучшие параметры: {study.best_params}")

best_params = study.best_params.copy()
best_params.update({
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'eval_at': [5, 10],
    'verbose': 1,
    'early_stopping_rounds' : 50
})

train_data_final, vali_data_final = create_datasets_with_params(feature_pre_filter=False)

final_model = train(
    best_params,
    train_data_final,
    valid_sets=[vali_data_final],
    valid_names=['vali'],
    num_boost_round=500,
)

vali_preds_final = final_model.predict(x_vali)
final_ndcg_5_vali = compute_fair_ndcg(y_vali, vali_preds_final, qid_vali, 5)
final_ndcg_10_vali = compute_fair_ndcg(y_vali, vali_preds_final, qid_vali, 10)

test_preds_final = final_model.predict(x_test)
tuned_ndcg_5_test = compute_fair_ndcg(y_test, test_preds_final, test_df['qid'], 5)
tuned_ndcg_10_test = compute_fair_ndcg(y_test, test_preds_final, test_df['qid'], 10)

print(f"\n=== ФИНАЛЬНЫЕ РЕЗУЛЬТАТЫ ===")
print(f"Vali NDCG@5: {final_ndcg_5_vali:.4f}")
print(f"Vali NDCG@10: {final_ndcg_10_vali:.4f}")
print(f"Test NDCG@5: {tuned_ndcg_5_test:.4f}")
print(f"Test NDCG@10: {tuned_ndcg_10_test:.4f}")


print(f"\n=== СРАВНЕНИЕ С BASELINE ===")
print(f"Улучшение Test NDCG@5: {tuned_ndcg_5_test - final_ndcg_5_test:.4f}")
print(f"Улучшение Test NDCG@10: {tuned_ndcg_10_test - final_ndcg_10_test:.4f}")



[I 2025-09-14 00:20:53,047] A new study created in memory with name: no-name-a8334845-1119-40de-8f3e-0bb8c4fdcb6d


Запуск оптимизации гиперпараметров...


  0%|          | 0/30 [00:00<?, ?it/s]

[I 2025-09-14 00:20:53,666] Trial 0 finished with value: 0.33966064560005893 and parameters: {'learning_rate': 0.017764180869738754, 'num_leaves': 44, 'max_depth': 6, 'min_child_samples': 33, 'subsample': 0.7765478326844456, 'colsample_bytree': 0.8751141179389358, 'reg_alpha': 0.23609675363713828, 'reg_lambda': 0.003516726510780027}. Best is trial 0 with value: 0.33966064560005893.
[I 2025-09-14 00:20:54,104] Trial 1 finished with value: 0.33962755669482025 and parameters: {'learning_rate': 0.04299350007173659, 'num_leaves': 69, 'max_depth': 5, 'min_child_samples': 45, 'subsample': 0.8222829810483582, 'colsample_bytree': 0.7544599639519277, 'reg_alpha': 0.0036464123892989016, 'reg_lambda': 0.0167838609572824}. Best is trial 1 with value: 0.33962755669482025.
[I 2025-09-14 00:20:57,426] Trial 2 finished with value: 0.34133491475540445 and parameters: {'learning_rate': 0.01585412471289983, 'num_leaves': 58, 'max_depth': 7, 'min_child_samples': 27, 'subsample': 0.8207021957438609, 'colsam

Подбор параметров не дал адекватного результата. Решил сделать кросс-валидацию



#Кросс-Валидация